<a href="https://colab.research.google.com/github/aid4mh/QPrism/blob/main/tests/Sensor/single_file_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! python3 -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


Install the dependencies

In [ ]:
! pip install -r https://raw.githubusercontent.com/aid4mh/QPrism/main/requirements.txt

In [3]:
! pip install --no-deps -i https://test.pypi.org/simple/ QPrism==0.1.6

Looking in indexes: https://test.pypi.org/simple/, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from QPrism.Sensor.DQM import DQM_single_file
from QPrism.Sensor.load_data import *

Load a sufficient json record and demonstrate the structure of the loaded data frame. For the original json file data structure, check the provided test data under data/sensor/json in the repo. modules/Sensor/load_data.py provides examples of how to load a json/csv file into this particular structure locally.

In [43]:
from urllib.request import urlopen
import json
import pandas as pd

response = urlopen("https://raw.githubusercontent.com/aid4mh/QPrism/main/data/sensor/json/SUBJ00001_Accelerometer_REC000000.json")
json_data = response.read().decode('utf-8', 'replace')

json_file = json.loads(json_data)
json_file = json.dumps(list(json_file['records']))
df = (pd.read_json(json_file))
df[list(df.keys())[0]] = df[list(df.keys())[0]].apply(lambda x: x.value)
print(df)

               timestamp         x         y          z
0    1593954057754000000  2.229002  6.114791   5.324705
1    1593954058125000000  2.135628  4.283227   6.833052
2    1593954058506000000 -1.721431  5.561730  10.201694
3    1593954058729000000 -0.220267  5.365406   7.668628
4    1593954058937000000 -0.177171  5.130774   8.346188
..                   ...       ...       ...        ...
440  1593954154639000000 -0.258574  6.212954   7.388506
441  1593954154858000000 -0.301669  6.212954   7.378930
442  1593954155076000000 -0.275333  6.193800   7.407660
443  1593954155276000000 -0.385466  6.205771   7.474698
444  1593954155491000000 -0.385466  6.215348   7.129933

[445 rows x 4 columns]


Create the DQM_single_file object, set the input data frame, then compute DQM

In [44]:
single_json_file_dqm = DQM_single_file()
single_json_file_dqm.set_input_data(df)
single_json_file_dqm.compute_DQM()

Get the fields of computed data quality matrix

In [7]:
single_json_file_dqm.get_fields()

['IRLR', 'SNR', 'VDR', 'SRC', 'MDR', 'APD']

Get the computed data quality matrix as a list, each element in the list represents a metric.

In [8]:
single_json_file_dqm.get_DQM()

['1',
 '-3.096340603605768',
 '1.0',
 '0.9011216710642995',
 '0.013303769401330377',
 '0.015730337078651686']

Get each single metric of the computed DQM as a str

In [9]:
single_json_file_dqm.get_IRLR()

'1'

In [10]:
single_json_file_dqm.get_SNR()

'-3.096340603605768'

In [11]:
single_json_file_dqm.get_VDR()

'1.0'

In [12]:
single_json_file_dqm.get_SRC()

'0.9011216710642995'

In [13]:
single_json_file_dqm.get_MDR()

'0.013303769401330377'

In [14]:
single_json_file_dqm.get_APD()

'0.015730337078651686'

We can also get the detected outliers' index

In [45]:
single_json_file_dqm.get_anomaly_index()

[268, 277, 306, 310, 334, 340, 354, 360, 361, 367, 382, 384]

Save the computed DQM as a csv file at given path.

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Change the saved path to your gdrive path before run the following lines.

In [18]:

single_json_file_dqm.save_to_file('/content/gdrive/MyDrive/CAMH/JOSS/Sensor_result/demo_result_single_file.csv')

Data successfuly saved.


Display the saved csv file.

In [19]:
with open("/content/gdrive/MyDrive/CAMH/JOSS/Sensor_result/demo_result_single_file.csv", "r") as DQM_csv:
    for row in DQM_csv:
        print(row)

IRLR,SNR,VDR,SRC,MDR,APD

1,-3.096340603605768,1.0,0.9011216710642995,0.013303769401330377,0.015730337078651686



Set SNR to be not included in DQM

In [20]:
single_json_file_dqm.set_SNR(False)

Compute and get the updated DQM, now SNR should not be included in DQM.

In [21]:
single_json_file_dqm.compute_DQM()

In [22]:
single_json_file_dqm.get_fields()

['IRLR', 'VDR', 'SRC', 'MDR', 'APD']

In [23]:
single_json_file_dqm.get_DQM()

['1',
 '1.0',
 '0.9011216710642995',
 '0.013303769401330377',
 '0.02696629213483146']

Now get_SNR should not be called

In [24]:
single_json_file_dqm.get_SNR()

'SNR is not computed according to the configuration'

Note that you can set any metrics except IRLR to be not included as above.



For an insufficient record, DQM will have np.NaN for all metrics except IRLR. IRLR will have a value of 0 to indicate the input record is insufficient. The following record is an empty record, thus the input data is an empty DataFrame.

In [26]:
response = urlopen("https://raw.githubusercontent.com/aid4mh/QPrism/main/data/sensor/json/SUBJ00001_Accelerometer_REC000006.json")
json_data = response.read().decode('utf-8', 'replace')

print(json_data)

In [30]:
df = pd.DataFrame([])
single_json_file_dqm.set_input_data(df)


First we need to set SNR to be included, since we previously excluded SNR.

In [31]:
single_json_file_dqm.set_SNR(True)

Then we compute the DQM for this insufficient record.

In [33]:
single_json_file_dqm.compute_DQM()


In [34]:
single_json_file_dqm.get_fields()


['IRLR', 'SNR', 'VDR', 'SRC', 'MDR', 'APD']

In [35]:
single_json_file_dqm.get_DQM()

['0', nan, nan, nan, nan, nan]

For the definition of insufficient, please check the IRLR in this [Glossary](https://qprism.readthedocs.io/en/latest/glossary.html#sensor-module).

Now load a sufficient csv sensor data, similarly, you can check modules/Sensor/load_data.py for the code that loads the csv into this structure locally.

In [36]:
df = load_data_csv_one_file('https://raw.githubusercontent.com/aid4mh/QPrism/main/data/sensor/csv/id9603e9c3.csv')
print(df)

         time_s   lw_x   lw_y   lw_z   lh_x   lh_y   lh_z   la_x   la_y  \
0          0.01 -0.004  0.945 -0.461  0.055  0.992 -0.199  0.168  1.418   
1          0.02 -0.406 -0.379  1.031 -0.320 -0.102  0.844 -0.422  0.559   
2          0.03 -0.340 -0.918  1.160 -0.152 -0.676  1.160 -0.500  1.004   
3          0.04 -0.207 -1.023  0.344  0.059 -0.797  0.453 -0.363  1.363   
4          0.05 -0.152 -1.098 -0.012  0.133 -0.875  0.133 -0.301  1.582   
...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
111295  1112.96  0.000  0.000  0.000  0.246  0.012  0.949  0.453  0.102   
111296  1112.97  0.000  0.000  0.000  0.246  0.012  0.949  0.449  0.102   
111297  1112.98  0.000  0.000  0.000  0.246  0.012  0.949  0.449  0.102   
111298  1112.99  0.000  0.000  0.000  0.246  0.012  0.949  0.449  0.102   
111299  1113.00  0.000  0.000  0.000  0.246  0.012  0.949  0.453  0.102   

         la_z   ra_x   ra_y   ra_z  
0      -0.070 -0.105  1.000  0.125  
1       0.891 -0.352 -0.0

Compute the DQM, this computation takes significantly longer time, since the record contains 111300 rows × 13 columns of data. The runtime for this record on a PC with intel i7-8700k processor, 16G DDR4 3600MHz RAM is ~160s

In [37]:
single_csv_file_dqm = DQM_single_file()
single_csv_file_dqm.set_input_data(df)
single_csv_file_dqm.compute_DQM()

Now display the result

In [38]:
single_csv_file_dqm.get_fields()

['IRLR', 'SNR', 'VDR', 'SRC', 'MDR', 'APD']

In [39]:
single_csv_file_dqm.get_DQM()

['1',
 '-3.436961042075442',
 '1.0',
 '0.9999999999986589',
 '0.0',
 '0.0029469901168014375']

In [41]:
single_csv_file_dqm.get_anomaly_index()

[103149,
 103271,
 103831,
 103878,
 104205,
 104260,
 104384,
 104506,
 104757,
 107901,
 107912,
 107940,
 107981,
 107991,
 108020,
 108050,
 108060,
 108064,
 108116,
 108124,
 108131,
 108134,
 108166,
 108170,
 108187,
 108188,
 108189,
 108238,
 108245,
 108246,
 108283,
 108284,
 108294,
 108299,
 108311,
 108313,
 108327,
 108329,
 108344,
 108390,
 108404,
 108451,
 108455,
 108462,
 108467,
 108470,
 108475,
 108501,
 108509,
 108527,
 108560,
 108561,
 108590,
 108600,
 108604,
 108617,
 108620,
 108641,
 108648,
 108650,
 108651,
 108654,
 108673,
 108690,
 108738,
 108752,
 108781,
 108783,
 108784,
 108788,
 108796,
 108826,
 108846,
 108847,
 108849,
 108851,
 108895,
 108897,
 108898,
 108904,
 108913,
 108954,
 108955,
 108963,
 108994,
 109036,
 109071,
 109076,
 109090,
 109092,
 109132,
 109169,
 109172,
 109177,
 109178,
 109192,
 109209,
 109255,
 109294,
 109301,
 109303,
 109312,
 109313,
 109317,
 109325,
 109348,
 109350,
 109351,
 109386,
 109401,
 109402,
 